In [1]:
import awswrangler as wr

from data_sci_toolkit.aws_tools import permission_tools 

/opt/miniconda3/envs/stellar/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
SESSION = permission_tools.get_aws_session("Dev-DataScienceAdmin")

Authorized as AROAQBUNWK7XN5DTYWPLJ:rkazmerik@stellaralgo.com


In [3]:
DATABASE = "integrations_milb_aviators"
S3_BUCKET = "s3://dev-athena-testing-us-east-1"

In [8]:
df = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """
        SELECT * FROM integrations_milb_aviators.tickets LIMIT 10 
    """
)
df.shape

(10, 26)

In [9]:
df

,source_system,source_system_group,product,client_property,seat_id,section_name,row_name,seat_num,season_id,event_id,...,price_code,price,account_rep_id,account_rep_name,purchased_at,returned_at,updated_at,forwards,resales,scans
0,ticketmaster,ticketing,Individual,milb_aviators,B03EB493E6F534EE8DF7EF285B2D0E7D,DOB107,K,4,79,3340,...,ESQ,18.590000,1243873,Kyle Nakama,2020-09-23 10:16:22.888,2021-03-03 10:37:37.007,2020-09-23 10:16:23,None,None,None
1,ticketmaster,ticketing,Half Season,milb_aviators,51F8EB01D3D33DE357D8615F3472531D,DOB114,S,9,79,3329,...,ESH,17.290001,31,TJ Thedinga,2020-11-11 09:36:45.592,2021-03-08 13:59:43.669,2020-11-11 09:36:45,None,None,None
2,ticketmaster,ticketing,Half Season,milb_aviators,B10CF27EF6CB1AF2569096BCE92F3365,DOB108,S,4,79,3335,...,ESH,17.289999,31,TJ Thedinga,2020-11-11 09:43:45.722,2021-03-08 13:59:38.692,2020-11-11 09:43:45,None,None,None
3,ticketmaster,ticketing,Individual,milb_aviators,F488F281815B68FA3CB96940371074A0,DOB108,W,12,79,3306,...,ESQ,18.590000,31,TJ Thedinga,2020-11-11 09:24:59.930,2021-03-08 14:24:30.836,2020-11-11 09:25:00,None,None,None
4,ticketmaster,ticketing,Full Season,milb_aviators,C1728CC1F9CEA7E6DA875467B27E1D4D,HPP111,E,6,79,3299,...,ASCH,32.500000,16,Erik Eisenberg,2021-02-27 08:15:15.048,2021-03-08 13:19:34.406,2021-02-27 08:15:15,None,None,None
5,ticketmaster,ticketing,Full Season,milb_aviators,9938179B79F9D69B6CCAFBB5C3752214,DOP108,H,15,79,3284,...,DSCH,19.650000,16,Erik Eisenberg,2020-11-16 16:36:57.945,2021-03-08 13:17:12.281,2020-11-16 16:36:58,None,None,None
6,ticketmaster,ticketing,Full Season,milb_aviators,AEAA4508F19CE02F67A2D8DE75AA2A07,S6,B,3,79,3290,...,LSF,59.529999,7,Chuck Johnson,2020-09-21 10:12:12.861,2021-02-24 10:20:07.278,2020-09-21 10:12:13,None,None,None
7,ticketmaster,ticketing,Full Season,milb_aviators,9CFF34028FAEFD8BE9ED579CEE4B2900,S6GA,GA0,3,79,3297,...,LSF,59.529999,7,Chuck Johnson,2020-09-21 10:12:12.861,2021-02-24 10:20:07.503,2020-09-21 10:12:13,None,None,None
8,ticketmaster,ticketing,Full Season,milb_aviators,8741925BD4DEE9316EA6030C6797C9BE,DOB117,D,16,79,3317,...,ESF,15.720000,1213561,Ariel Greenberg,2021-03-03 10:16:41.435,2021-03-08 13:18:53.988,2021-03-03 10:16:41,None,None,None
9,ticketmaster,ticketing,Full Season,milb_aviators,A801FF76E8867C367027E9846A0EC618,HPB111,W,20,79,3332,...,CSCH,19.650000,1199851,Nathan Erbach,2020-10-21 12:05:08.285,2021-03-08 13:18:19.064,2020-10-21 12:05:08,None,None,None


In [ ]:
# REDSHIFT US-PROD
# DYNAMO DB - CONFIG